# Notebook for exploring astro imaging setups
by J.A. de Vries

In [1]:
import math

In [2]:
clight = 3.00e8     # m/s
hplanck = 6.626e-34 # J/Hz
au = 150e6   * 1e3
ly = 9.46e12 * 1e3
pc = 30.9e12 * 1e3

In [3]:
def rad_to_amin(rad) :
    return rad / (2 * 3.14) * 360 * 60

def amin_to_rad(amin) :
    return amin / 60 / 360 * (2 * 3.14)

def deg_to_degminsec(deg) :
    d = int(deg)
    m = (deg % 1) * 60
    s = (m % 1) * 60
    m = int(m)
    s = round(s)
    return [d, m, s]
    
def degminsec_to_deg(d, m, s) :
    return d + (m + s/60) / 60

def getPhotonsFromMag(mag) :
    # From rel. Mag to Flux:  (m - mref) = -2.5 log10(F/Fref)
    Fref = 3631   # Jy in V band for mref=0 [Vega / blackbody 11000 K] (3640?)
    flux = pow(10, mag / (-2.5)) * Fref
     
    # From flux to integrated spectral radiance (intensity)
    # 1 Jy = 10e-26 [W m-2 Hz-1], where m2 is surface on earth (d taken into account)
    wlrange = [400e-9, 700e-9]  # assume the same flux over all observed wavelengths
    hzrange = [ clight / wlrangei for wlrangei in wlrange ]
    I = flux * (hzrange[0] - hzrange[1]) * 10e-26 # intensity [W m-2]
    
    # From intensity to photons [# s-1 m-2]
    Eph = hplanck * clight / 550e-9 # assume green
    Nphotons = I / Eph
    return int(Nphotons)

### Classes

In [4]:
#https://www.photonstophotos.net/Charts/RN_e.htm#Canon%20EOS%20R_14,Nikon%20Z%206_14 
#https://www.cloudynights.com/topic/788719-some-canon-eos-r-technical-testing/ 

class Camera :
    def __init__(self, pixels, pixelsize, qe=0, adc=0, wd=0, rn=0, dc=0, bayer=0) :
        self.pixels    = pixels       # [x,y]
        self.pixelsize = pixelsize    # um
        #self.qe = qe # quantum efficiency [R,G,B]
        #self.welldepth = wd # full well-depth or adc bits?
        
        self.sensordims = [ round(pixelsi * self.pixelsize * 1e-3,1) for pixelsi in self.pixels ] # mm
        
        #self.readnoise = 2**1.2  # e-/pixel, goes down with ISO, R: [800 = 2**1.6, 1600 = 2**1.2, 3200 = 2**1, 6400 = 2**0.8
        #self.dcnoise   = 0.15    # e-/pixel per second

        # self.shotnoise?
        # conversion noise?
        # add full well capacity and dynamic range calculations for max. exposure time of 1/3 histo etc.?
        # add QE and gain/iso?
        # add filter factor (in)efficiency? (different response to signal vs noise)

    
class Lens :
    def __init__(self, focallength, f) :
        self.focallength = focallength # mm
        self.f           = f
        
        self.aperture = round(self.focallength / self.f,1) # mm
        self.area = round(3.14 * pow(self.aperture/2,2), 1) # mm2
        
    def airydisks(self) : 
        # more detailed airy disk deliberations: https://www.astropix.com/html/astrophotography/astrophotography-formulae.html
        return [2.44 * l * self.f * 1e-3 for l in [700, 550, 400] ] # diffraction-limited sharpness diameter [um]
        
    def dawes(self) :
        # more 'resolving power' definitions exist, e.g. Rayleigh.
        return 11.6 / (self.aperture * 0.1) # target resolving power in arcseconds.

                
        
class Filter :
    # Hga 434.0nm, Hb 486.1nm, OIII 500.7nm, NII (654, 658nm), Ha 656.3nm, SII 672nm
    # F-stops: double/half light, : 2.8, 4.0, 5.6, 8.0, 11.0
    def __init__(self, passbands) :
        self.passbands = passbands # [ [[passband_low, passband_high], eff] ]
        
    def passFraction(self) :
        fullrange = [400,700]
        fullint = (fullrange[1]-fullrange[0])*1.
        partint = 0
        for passband in self.passbands :
            partint += (passband[0][1] - passband[0][0]) * passband[1]
        return partint/fullint
    
    def signalEff(self) :
        # assume signal fully falls in the passbands. (Bad choice of filter on target is not included!)
        partint, partnorm = 0
        for passband in self.passbands :
            partint += (passband[0][1] - passband[0][0]) * passband[1]
            partnorm += (passband[0][1] - passband[0][0])
        return partint / partnorm    
        
        
class Mount :
    def __init__(self, pererr) :
        self.pererr = pererr # periodic error [arcsec/5 min]

        
class Site :
    def __init__(self, sqm, lat, long, seeing = 2.5, timedate=0) :
        self.latitude  = degminsec_to_deg(*lat)  # degrees N/S from equator, [-90, 90]
        self.longitude = degminsec_to_deg(*long) # degrees E/W from prime meridian, 
        self.seeing    = seeing # Seeing conditions in arcsec. 1.0 is great, 5.0 is bad. Check meteoblue for day-to-day values.
        self.timedate  = timedate # [hour, day, month, year]
        
        self.sqm  = sqm # mag/arcsec2
        # use https://www.lightpollutionmap.info/, SQM from World Atlas 2015.
        # World atlas is earth-based sky quality measure, VIIRS is sattelite-based observations (trans., not albedo).
        # B7=[18.0-18.5], B6=[18.5-19.3], B5=[19.3-20.3], B4=[20.8-21.3], B3=[21.3-21.6]. Fully dark site: B1 = 21.8.
        
        def getLPPhotons(self) :
            return
            
        #def getLostFraction
            

class Target :
    def __init__(self, mag, size, ra, dec) :
        self.mag  = mag  # observed magnitude
        self.size = size # [arcmin, arcmin]
        self.ra   = ra   # right ascension in h [0-24]
        self.dec  = dec  # declination in degrees [-90,90]

        #self.dist = X / pc
        #self.abssize = [ math.tan(amin_to_rad(sizei)) * self.dist / ly for sizei in self.size ] # in [ly,ly]
        
    def getPhotons(self) :
        return getPhotonsFromMag(self.mag)
                

### Functions of multiple classes

In [5]:
def skycoverage( cam, lens, pr=1 ) :
    # Returns sky coverage of camera setup in [arcmin x arcmin]
    cov = [ math.atan( sensordim / lens.focallength ) for sensordim in cam.sensordims ]
    cov = [ round(rad_to_amin(covi), 1) for covi in cov ]
    if(pr) : print("Camera + lens combo covers {} x {} arcmin of the sky".format(*cov))
    return cov


def targetcoverage( cam, lens, target, pr=1 ) :
    # Returns fraction of sensor covered by target
    cov = skycoverage(cam, lens, 0)
    scov = [target.size[0]/cov[0]*100., target.size[1]/cov[1]*100.]
    if(pr) : print("Target covers {:.0f}% x {:.0f}% of sensor".format(*scov))
    return scov


def imagescale( cam, lens, pr=1 ) :
    # arcsec / pixel
    # TODO calculate 206 factor.
    imscale = 206.3 * cam.pixelsize / lens.focallength
    if(pr) : print("Image scale = {:.2f} arcsec / pixel".format(imscale))
    return imscale

def calcSeeing( cam, lens, site ) :
    # oversampling:  [long focal,  small pixels, low arcsec/pixel]  -> soft/bloaty stars.  
    #    -> need larger expo to cross S/N thresh., but good setup for very clear nights -> tools: bin 2x2, or use deconv. 
    # undersampling: [short focal, large pixels, high arcsec/pixel] -> blocky star shapes.
    #    -> quick to cross S/N barrier, but in general not recommended, hard to recover -> tools: drizzle stacking.
    # "calculated resolution should be 1/2 of seeing conditions". Better seeing = more undersampled.
    # Seeing conditions: the smallest size you can still resolve due to atmospheric turbulence / humidity
    imscale = imagescale( cam, lens, pr=0 )
    diff = imscale - site.seeing/2
    overunder = "perfectly sampled"
    if diff > 0 : overunder = "undersampled by {:.1f}\"/px  (favouring SNR over resolution)".format( diff)
    if diff < 0 : overunder =  "oversampled by {:.1f}\"/px  (favouring resolution over SNR)".format(-diff)
    #print("Ideal seeing conditions for setup: {:.1f}\"".format(imscale*2))
    print("For your site seeing conditions ({:.1f}\") you are {}".format(site.seeing, overunder))
    if diff >  0.5: print(" -> Recommend dithering & using drizzle when stacking")
    if diff < -0.5: print(" -> Recommend deconvolution to sharpen images in post")
    if imscale <= site.seeing/4: print(" -> Recommend binning your cam sensor in 2x2")
    

def resolving( cam, lens, target=0, pr=1 ) :
    # maximum 'focus' size when diffraction limited
    airydisks = lens.airydisks()
    pixelsize = cam.pixelsize
    airypixels = [airydisk/pixelsize for airydisk in airydisks]
    if(pr) : print("Diffraction-limited focus for R,G,B is {:.1f}, {:.1f}, {:.1f} pixels".format(*airypixels))
    
    # target angular resolving power. (Eye: ~2 arcmin)
    dawes = lens.dawes()
    dawespixels = dawes / imagescale( cam, lens, 0 )
    if(pr) : print("Lens Dawes resolving limit: {:.1f} arcsec. This is {:.1f} pixels on sensor.".format(dawes, dawespixels))
    if(target) :
        dawestargetfrac = [dawes / (targetsizei * 60) * 100. for targetsizei in target.size]
        if(pr) : print(" -> This is {:.3f}% x {:.3f}% of your target".format(*dawestargetfrac))
        
    return 
    

def trackingerror( cam, lens, mount, pr=1 ) :
    scale = imagescale(cam, lens, 0 ) # arcsec/pixel
    pererr = mount.pererr / (5) # arcsec/min
    error = pererr / scale # pixel/min
    if(pr) : print("Tracking error: {:.2f} pixels/min. For ~1px shift you can expose for {:.0f} s".format(error, 60./error))
    return error


def checkGuideCompatibility( cam, lens, guidecam, guidescope, pr=1 ) :
    imscaleMain  = imagescale( cam, lens, 0 )
    imscaleGuide = imagescale( guidecam, guidescope, 0 )
    ratio = imscaleGuide / imscaleMain
    print("Guide / main scale = {:.1f}".format(ratio))
    if(ratio < 3) :  print("Guide scale considered good.")
    elif(ratio < 4) : print("Guide scale considered reasonable.")
    elif(ratio >= 4) : print("Warning: guide scale considered too course for main setup")
    # ratio of 4 -> based on what?
    
    centroidsubpixelprecision = 0.1   # "0.1-0.15 for single star, 0.05 for multi-star guiding"
    arcsecguiding = centroidsubpixelprecision * imscaleGuide
    print(" -> With sub-px centroid accuracy of {}, expected PHD2 guiding precision = {:.2f} arcsec."\
          "\n    This is {:.2f} th of a pixel of your system.".format(
          centroidsubpixelprecision, arcsecguiding, arcsecguiding / imscaleMain) )
    return

    
def sigPhotonsPerPixel( target, lens, cam, pr=1 ) :
    # returns signal photons per pixel per second
    # assumes filter = 100% eff.
    Nphotons = target.getPhotons() # per m2 per s
    lensarea = lens.area * 1e-6
    Nphotons_in_lens = Nphotons * lensarea
    if(pr): print("Total target signal photons in lens: {:.0f} M /s".format(Nphotons_in_lens / 1e6))
    
    # Assume target is projected is completely inside sensor, and target brightness is uniform
    targetcov = targetcoverage(cam, lens, target, 0) # in %
    pixeldims = cam.pixels
    signalpixels = [a*b / 100 for a, b in zip(targetcov, pixeldims)]
    signalpixels = signalpixels[0] * signalpixels[1]
    photonsperpixel = Nphotons_in_lens / signalpixels
    if(pr) : print("Target signal photons per pixel: {:.1f} /s".format(photonsperpixel))
    return photonsperpixel
    # Add photons lost fraction due to sky scattering, depending on travel length through atmosphere. Need rel. sky position for that.

    
def skybkgPerPixel( site, camera, lens, filtr=0 ) :
    # Backwards engineer from http://www.tools.sharpcap.co.uk
    # Info at https://www.cloudynights.com/topic/553640-convert-radiance-into-magnitudearcsec2/ 
    # world atlas 2015: SQM = 18.84 mag/arcsec^2 = Bortle class 7. 
    # viirs 2015: radiance 22.30 [10-9 W / cm^2 sr]
    # VIIRs measures 'light on earth from sattelite', but SQM-L measures 'brightness of sky' = pollution.
    # So take from World Map SQM, but scale with VIIRS light pollution change between 2015 and now.
    return

def polarAlignmentDrift( camera, lens, target, location ) :
    return
    # Star Adventurer polar alignment: <6 arcmin. "Good polar alignment for long-term exposures" is <1 arcmin, achievable with "3-position plate solving" software like iPolar, Ekos, etc.
    # But what is the actual effect in pixels drifted per minute? 
    # Depends on polar misalignment from mount (in both axes) and arcsec/pixel of the camera/lens setup.
    # Also on where the target is in the sky.
    # Drift is mostly in Dec axis.
    
def optimalExposureTime() :
    return
    # Depends on when signal rises above the (read) noise floor. 
    # Longer = more challenging process in general - tracking/guiding, impact of wrong subs, less frames for drizzle (but less disk space).
    # Modern sensors have very low read noise, so "not much of an issue anymore".
    # Typical fora values: "under the Bortle 3 skies where I live that the ultimate best exposure to record the faintest detail possible was 96 seconds for broadband, 
    #  two minutes for Oiii and five minutes for H-alpha." (calibrated with sharpcap pro)

### Pre-defined setup data

In [6]:
Cameras = {
    "canon_eosR" : dict(pixels = [6720, 4480], pixelsize = 5.36, qe = [0.20, 0.70, 0.75], adc=14, wd=64e3, rn=1.4+2.9, dc = 0.12, bayer="RGGB"), # full-frame. rn+convnoise? at 1600 iso. Note: QE is not absolute, see https://www.cloudynights.com/topic/788719-some-canon-eos-r-technical-testing/
    "canon_5DmkII" : dict(pixels = [5616, 3744], pixelsize = 6.41), # full-frame but with some experience
    "zwo_1600mc" : dict(pixels = [4656, 3520], pixelsize = 3.80, qe = [0.50, 0.60, 0.40], adc=12, wd=20e3, rn=1.2, bayer="RGGB"), # #TODO rn vs T - pick T.
    "zwo_1600mm" : dict(pixels = [4656, 3520], pixelsize = 3.80, qe = [0.50, 0.60, 0.50], adc=12, wd=20e3, rn=1.2, bayer="MONO"), # TODO rn at certain temp?
    "zwo_2600mc" : dict(pixels = [6248, 4176], pixelsize = 3.76, qe = [0.57, 0.75, 0.80], adc=16, wd=50e3, rn=1.0, bayer="RGGB"), # aps-c. No amp glow vs 1600's.
    "zwo_2600mm" : dict(pixels = [6248, 4176], pixelsize = 3.76, qe = [0.65, 0.85, 0.91], adc=16, wd=50e3, rn=1.0, bayer="MONO"), # aps-c, higher QE / fullwelldepth 2.5x, 16bit ADC for dynrange, no amp glow.
    "zwo_120mm"  : dict(pixels = [1280, 960],  pixelsize = 3.75), # the 'goto' cheap default guiding cam
    "zwo_290mm"  : dict(pixels = [1936, 1096], pixelsize = 2.90),
    "zwo_220mm"  : dict(pixels = [1920, 1080], pixelsize = 4.00), # new upgrade of 290
    "zwo_174mm"  : dict(pixels = [1936, 1216], pixelsize = 5.86), # ideal for OAG light but expensive
}


Filters = {
    "antlia_tri"  : dict(passbands = [[[430,445], 0.92], [[490,520], 0.95], [[650,675], 0.98]]), # Antlia 'Triband RGB Ultra', 'broad narrowband', Hga, OIII, Ha, SII. [Alt: antlia quad, idas GNB/DTD]
    "opto_lenh"   : dict(passbands = [                   [[485,510], 0.93], [[652,660], 0.93]]), # Optolong L-Enhance, broad Hb + OIII, narrow Ha. Alternatives: ZWO duo, IDAS NB1, Askar duo.
    "idas_nbz"    : dict(passbands = [                   [[494,505], 0.96], [[651,664], 0.96]]), # IDAS NBZ 'Nebula Boost' 10nm bp OIII, Ha. Broad but high eff and no halos. Good for fast.  
    "opto_lext"   : dict(passbands = [                   [[497,504], 0.91], [[653,660], 0.91]]), # Optolong L-Extreme  dual 7nm bp OIII, Ha # warning: extreme halos. 
    "antlia_alpt" : dict(passbands = [                   [[498,503], 0.81], [[654,659], 0.90]]), # Antlia ALP-T        dual 5nm bp OIII, Ha # no haloes. ('fast' version produces halos on red stars) 
    "opto_lult"   : dict(passbands = [                   [[499,502], 0.90], [[655,658], 0.90]]), # Optolong L-Ultimate dual 3nm bp OIII, Ha # warning: bandpass shift < f/4, and some halos. 
}


Lenses = {
    "canon_EF400"      : dict(focallength =  400, f =  5.6), # Compare to [WO redcat 71 350mm f/4.9] [Zenithstar 73-III 430mm f/5.9] [AT70ED 420mm f/6]
    "canon_EF400_1.4"  : dict(focallength =  560, f =  8.0), # EF 1.4x extender, <300.-. Compare to [SV503 80ED 560mm f/7 FPL51 doublet] [ZWO FF80 600mm f/7.5] [SW evostar AP80 600mm f/7.5 FPL53 doublet]
    "canon_EF200"      : dict(focallength =  200, f =  4.0),
    "canon_EF70"       : dict(focallength =   70, f =  4.0),
    "sigma_150"        : dict(focallength =  150, f =  5.0),
    "sigma_600"        : dict(focallength =  600, f =  6.3),
    "askar_107PHQ"     : dict(focallength =  749, f =  7.0), # 3000.- (opt 0.7x reducer for 550.- -> 525mm @ f/4.9)
    "askar_120APO"     : dict(focallength =  840, f =  7.0), # 2000.- + 335.- (flat) w=6.5kg
    "askar_120APO_0.8" : dict(focallength =  672, f =  5.6), # 0.8x reducer, + 335.-
    "askar_130PHQ"     : dict(focallength = 1000, f =  7.7), # 4000.-
    "celest_edgeHD8"   : dict(focallength = 2032, f =  10.), # # 2200.-  , w=6.35kg, 'light gathering power 843x human eye'
    "sw_evoguide50ED"  : dict(focallength =  242, f =  4.8),
}


Mounts = {
    "sw_staradventurer"        : dict(pererr = 30),   # w. lim. 5kg
    "sw_staradventurer_guided" : dict(pererr = 1.5),
    "ioptron_CEM40"            : dict(pererr = 5),    # w. lim. 18kg 
    "ioptron_CEM40_guided"     : dict(pererr = 0.5),
}


Targets = {
    # Galaxies
    "M31"     : dict(mag =  3.28, size = [177.8,  69.7], ra= 1, dec=+41), # Andromeda
    "M33"     : dict(mag =  5.79, size = [ 62.1,  36.7], ra= 2, dec=+31), # Triangulum [close to Andromeda]
    "M81"     : dict(mag =  6.77, size = [ 21.6,  11.2], ra=10, dec=+69), # Bode's galazy [Ursa Major]
    "M51"     : dict(mag =  7.92, size = [ 13.7,  11.7], ra=13, dec=+47), # Whirlpool [Ursa Major]
    "M101"    : dict(mag =  7.77, size = [ 24.0,  23.1], ra=14, dec=+54), # Pinwheel [Ursa Major]
    "M104"    : dict(mag =  8.12, size = [  8.4,   4.9], ra=13, dec=-11), # Sombrero [Virgo]
    "M64"     : dict(mag =  8.38, size = [  10.5,  5.3], ra=13, dec=+22), # Black eye [near Arcturus]
   #"M86"     : dict(mag =  8.79, size = [ 11.5,   8.4], ra=12, dec=+13), # M86 galaxy [Virgo]
    "Markch"  : dict(mag =  8.79, size = [ 96.0,  54.0], ra=12, dec=+13), # Markarian's chain (M86, M84, ...) [Virgo]
    "NGC6946" : dict(mag =  8.88, size = [ 11.4,  10.8], ra=21, dec=+60), # Fireworks, lot of Ha [~near Deneb]
   #"M66"     : dict(mag =  8.91, size = [ 10.3,   4.6], ra=11, dec=+13), # M66 [Leo]
    "Leotrip" : dict(mag =  8.91, size = [ 42.0,  42.0], ra=11, dec=+13), # Leo triplet (M66, M65, NGC3628)
    "NGC2841" : dict(mag =  9.12, size = [  6.9,   3.3], ra= 9, dec=+51), # unbarred sideways spiral [near Ursa Major]

    # Star formations
    "Hyades"  : dict(mag =  0.50, size = [330.0, 330.0], ra= 4, dec=+15), # Hyades (Aldebaran + head of Taurus) [close to Pleiades]
    "Polaris" : dict(mag =  2.00, size = [  0.0,   0.0], ra= 3, dec=+89), # 'North' reference
    "M45"     : dict(mag =  1.50, size = [120.0, 120.0], ra= 4, dec=+24), # Pleiades, + refl.
    "coath"   : dict(mag =  3.59, size = [ 89.0,  89.0], ra=19, dec=+20), # Coat hanger / Brocchi's cluster
    "M24"     : dict(mag =  4.59, size = [ 90.0,  90.0], ra=18, dec=-18), # Saggitarius star cloud [Milky way]
    "M13"     : dict(mag =  5.78, size = [ 10.0,  20.0], ra=17, dec=+36), # Hercules cluster

    # SNR
    "cygloop" : dict(mag =  5.00, size = [180.0, 180.0], ra=20, dec=+31), # Veil nebulae (NGC6960, NGC6992), aka cygnus loop / witch broom [Cygnus], O-III
    "M1"      : dict(mag =  8.39, size = [  6.0,   4.0], ra= 5, dec=+22), # Crab
    "IC443"   : dict(mag = 12.00, size = [ 50.0,  40.0], ra= 6, dec=+22), # Jellyfish
    
    # Reflection nebulae
    "IC4592"  : dict(mag =  3.90, size = [150.0,  60.0], ra=15, dec=-19), # Blue horsehead [Scorpio, close to Antares]
    "NGC7023" : dict(mag =  7.19, size = [ 18.0,  18.0], ra=21, dec=+68), # Iris
    "IC2118"  : dict(mag = 10.00, size = [180.0,  60.0], ra= 5, dec= -8), # Witch head [near Rigel], SNR
   #"Sh2-136" : dict(mag =  0   , size = [  0  ,   0  ], ra=21, dec=+68), # Ghost, dark [near Iris]. 

    # Mixed diffuse nebulae (reflection/emission/dark)
    "NGC2264" : dict(mag =  3.90, size = [ 60.0 , 30.0], ra= 7, dec=+10), # Cone nebula & Christmas tree cluster [close to Betelgeuse]
    "M42"     : dict(mag =  4.00, size = [ 85.0,  60.0], ra= 5, dec= -5), # Orion
    "tandl"   : dict(mag =  4.59, size = [ 90.0, 180.0], ra=18, dec=-23), # Trifid & Lagoon [Milky way]
    "M20"     : dict(mag =  6.30, size = [ 29.0,  27.0], ra=18, dec=-23), # Trifid [Milky way, close to Lagoon]
    "IC4604"  : dict(mag =  5.09, size = [ 60.0,  25.0], ra=16, dec=-24), # Rho Ophiuchi complex
    "antcom"  : dict(mag =  5.09, size = [200.0, 180.0], ra=16, dec=-24), # Antares + Rho Ophiuchi complex
   #"SH2-129" : Flying bat & squid nebulae
   #"B150"    : Barnard 150 aka Seahorse, dark nebula.
    
    # Mixed composition emission nebulae
    "NGC7000" : dict(mag =  4.00, size = [120.0, 100.0], ra=21, dec=+45), # North America [close to Deneb, + Pelican]
    "NGC2244" : dict(mag =  5.50, size = [ 80.0,  60.0], ra= 7, dec= +5), # Rosette [near Betelgeuse]
    "IC1396"  : dict(mag =  5.59, size = [ 35.6,  35.6], ra=22, dec=+57), # Elephant trunk nebula [close to Deneb]
    "eltrunk" : dict(mag =  5.59, size = [170.0, 140.0], ra=22, dec=+57), # Complete elephant trunk
    "M17"     : dict(mag =  6.00, size = [ 46.0,  37.0], ra=18, dec=-16), # Omega [Milky way]
    "M16"     : dict(mag =  6.40, size = [ 35.0,  28.0], ra=18, dec=-14), # Eagle [Milky way]
    "IC1805"  : dict(mag =  6.50, size = [ 60.0,  60.0], ra= 3, dec=+61), # Heart [close to Cassiopeia]
    "hands"   : dict(mag =  6.50, size = [130.0,  70.0], ra= 3, dec=+61), # Heart & Soul together
    "NGC7380" : dict(mag =  7.19, size = [ 25.0,  30.0], ra=23, dec=+58), # Wizard, rich in O-III
    "NGC6888" : dict(mag =  7.40, size = [ 20.0,  10.0], ra=20, dec=+38), # Cresent, rich in O-III, fueled by WR136 [close to Sadr]
    "NGC2359" : dict(mag = 11.50, size = [  8.0,   6.0], ra= 7, dec=-13), # Thor's helmet / Duck, fueled by WR7
   #"WR134"   : dict(mag =  0   , size = [  0  ,   0  ], ra=20, dec=+36), # Unknown nebula data. Fueled by WR134, rich in O-III [Cygnus]
    
    # H-dominated emission nebulae
    "M8"      : dict(mag =  4.59, size = [ 90.0,  40.0], ra=18, dec=-24), # Lagoon [Milky way, close to Trifid]
    "NGC1499" : dict(mag =  5.00, size = [145.0,  40.0], ra= 4, dec=+36), # California [close to Pleiades]
    "IC1848"  : dict(mag =  6.50, size = [ 60.0,  30.0], ra= 3, dec=+60), # Soul [close to Heart]
    "NGC281"  : dict(mag =  7.40, size = [ 35.0,  30.0], ra= 1, dec=+57), # Pacman [close to Cassiopeia]
    "NGC2024" : dict(mag = 10.00, size = [ 30.0,  30.0], ra= 6, dec= -2), # Flame [Orion]
    "handf"   : dict(mag = 10.00, size = [108.0,  60.0], ra= 6, dec= -2), # Horsehead (dark) & Flame & NGC2023 (refl.) [Orion]
    "IC1318"  : dict(mag = 10.00, size = [ 40.0,  20.0], ra=20, dec=+42), # Gamma Cygni [Sadr]
    "IC405"   : dict(mag = 10.00, size = [ 30.0,  19.0], ra= 5, dec=+34), # Flaming star, (+ refl.), nearby 'tadpoles' IC417, IC410 [close to Pleiades]
   #"SH2-119", "clamshell". Unknown nebula data. In Cygnus, close to North America.
    
    # Planetary nebulae
    "M27"     : dict(mag =  7.09, size = [  8.0,   5.7], ra=20, dec=+23), # Dumbbell
    "NGC7293" : dict(mag =  7.59, size = [ 14.7,  12.0], ra=22, dec=-21), # Helix
    "M57"     : dict(mag =  8.80, size = [  1.4,   1.1], ra=19, dec=+33), # Ring    
    
    # Planetary objects
    "Moon"    : dict(mag = -10.7, size = [30.0, 30.0]),
}


Sites = {
    "Backyard" : dict(sqm = 18.84, lat = [52,  5, 20], long = [5,  3, 53], seeing=2.0),
    "Waggie"   : dict(sqm = 19.42, lat = [51, 59,  4], long = [5, 39,  1], seeing=2.0),
    "Puk"      : dict(sqm = 21.30, lat = [52, 56, 36], long = [6, 48, 46], seeing=2.0),
}


# Now define your setup

In [7]:
cam1 = Camera( **Cameras["canon_eosR"] )
#cam2 = Camera( **Cameras["zwo_2600mm"] )
cam2 = cam1

lens1 = Lens( **Lenses["canon_EF400"] )
lens2 = Lens( **Lenses["askar_120APO"] )
#lens = Lens( **Lenses["celest_edgeHD8"] )

filtr = Filter( **Filters["antlia_tri"] )

mount1 = Mount( **Mounts["sw_staradventurer"] )
mount2 = Mount( **Mounts["ioptron_CEM40"] )

guidecam = Camera( **Cameras["zwo_120mm"] )
guidescope = Lens( **Lenses["sw_evoguide50ED"] )

target = Target( **Targets["M42"] )
#target = Target( **Targets["NGC7293"] )

site = Site( **Sites["Backyard"] )

### And calculate useful quantities

In [8]:
print("Setup 1:")
skycoverage(cam1, lens1);
targetcoverage(cam1, lens1, target);

print("\nSetup 2:")
skycoverage(cam2, lens2);
targetcoverage(cam2, lens2, target);

Setup 1:
Camera + lens combo covers 308.7 x 206.1 arcmin of the sky
Target covers 28% x 29% of sensor

Setup 2:
Camera + lens combo covers 147.3 x 98.2 arcmin of the sky
Target covers 58% x 61% of sensor


In [9]:
print("Setup 1:")
resolving(cam1, lens1, target);

print("\nSetup 2:")
resolving(cam2, lens2, target);

Setup 1:
Diffraction-limited focus for R,G,B is 1.8, 1.4, 1.0 pixels
Lens Dawes resolving limit: 1.6 arcsec. This is 0.6 pixels on sensor.
 -> This is 0.032% x 0.045% of your target

Setup 2:
Diffraction-limited focus for R,G,B is 2.2, 1.8, 1.3 pixels
Lens Dawes resolving limit: 1.0 arcsec. This is 0.7 pixels on sensor.
 -> This is 0.019% x 0.027% of your target


In [10]:
print("Setup 1:")
trackingerror(cam1, lens1, mount1);

print("\nSetup 2:")
trackingerror(cam2, lens2, mount2);

Setup 1:
Tracking error: 2.17 pixels/min. For ~1px shift you can expose for 28 s

Setup 2:
Tracking error: 0.76 pixels/min. For ~1px shift you can expose for 79 s


In [11]:
print("Main setup:")
imagescale(cam1, lens1);

calcSeeing(cam1, lens1, site);

print()

print("Guide setup:")
imagescale(guidecam, guidescope);

checkGuideCompatibility( cam1, lens1, guidecam, guidescope );

Main setup:
Image scale = 2.76 arcsec / pixel
For your site seeing conditions (2.0") you are undersampled by 1.8"/px  (favouring SNR over resolution)
 -> Recommend dithering & using drizzle when stacking

Guide setup:
Image scale = 3.20 arcsec / pixel
Guide / main scale = 1.2
Guide scale considered good.
 -> With sub-px centroid accuracy of 0.1, expected PHD2 guiding precision = 0.32 arcsec.
    This is 0.12 th of a pixel of your system.


In [12]:
print("Setup 1:")
sigPhotonsPerPixel( target, lens1, cam1 ); 

print("\nSetup 2:")
sigPhotonsPerPixel( target, lens2, cam2 ); 

Setup 1:
Total target signal photons in lens: 32 M /s
Target signal photons per pixel: 13.5 /s

Setup 2:
Total target signal photons in lens: 92 M /s
Target signal photons per pixel: 8.6 /s


In [13]:
print("Filter lets through {:.1f}% of visible spectrum".format(filtr.passFraction() * 100.))

#skybkgPerPixel(site, camera, lens, filtr) 

Filter lets through 22.3% of visible spectrum


In [14]:
# Read noise goes down with higher ISO. (Why?)
# Longer exposures, read noise stays constant. But gets swamped by sky bkg.
# Shot noise of target & sky

### Notes

In [15]:
# dynamic range: eye can see '20 stops'. normal good camera at iso100 under 'normal lighting': 15 stops.
    # colour depth bits vs ISO: 24 bit at ISO 100, 20 at ISO 1000, 19 at ISO 1600, 15 at ISO 12800
    # https://www.dxomark.com/canon-eos-r-sensor-review/
    
    # eos R has 14-bit 'raw', = 2^14 'well depth'?
    # dynamic range = full well / read noise, to be captured (=should match) 2^ADC ?
    
    #https://astronomy-imaging-camera.com/product/asi2600mm-pro-mono/ 
#http://tools.sharpcap.co.uk
    
# advantage of mono:
# - "RGB": less focus on green, balanced. Need 3x sessions but record 4x more R,B data. Flexible.
#   -> OSC: 1h x (1/4 + 1/4 + 2/4) ; RGB: 1/3h x (1/3 + 1/3 + 1/3).
# - "LRGB" Luminence approach saves time with far better structure contrast. (but at cost of colour fidelity of you do eg 90% L).
#   -> total light in 1h: LRGB = (0.7h x 1 + 3* (0.1 * 1/3) = 0.8.   While RGB = 3*(1/3h * 1/3) = 0.33.
# - Huge improvement with narrowband data efficiency. Allows for creative HSO palettes etc.
# - No debayering interpolation, x2 (x4) resolution per color. (But after proper CFA drizzling with dithered OSC subs, helps in case of 'undersampled'. Though it can introduce extra noise.
# - Higher 'color filter' efficiency compared to bayer filters? (depends on quality filters)
# - Better colour separation compared to wide bayer filters. No Ha 'bleed' to green/blue.
# - Higher QE? (debatable - same sensor?)


# Add 'Seeing", over/undersampling (arcsec / pixel between 1 and 3? lower is oversampling but can be corrected by convolution, undersamp (= very small pixels with huge telescope) hitting the 'atmospheric seeing' limit is just a shame.
# but what is typical 'seeing'? 'good seeing' is 1.0", 'bad seeing' is 5.0". so under 'average seeing', 1.5" to 3.0" per pixel is recommended.
# "FWHM" of a star - and have at least 3 pixels fit in that. ('oversampling is not so bad').
#  Oversampling means SNR per pixel goes slower, waste of time. Undersampling means you don't get the best resolution for your objects.